In [ ]:
# Install required packages
!pip install smolagents litellm duckduckgo-search ddgs --quiet
print("✅ Packages installed!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 53.2 MB/s eta 0:00:00
✅ Packages installed!


In [ ]:
# API Key Setup
import os
from getpass import getpass

if 'GROQ_API_KEY' not in os.environ:
    os.environ['GROQ_API_KEY'] = getpass('Enter your Groq API key: ')

print("✅ API key configured!")

Enter your Groq API key: ··········
✅ API key configured!


In [ ]:
# Import libraries
from smolagents import tool, ToolCallingAgent, LiteLLMModel, DuckDuckGoSearchTool
import litellm
import json

MODEL_ID = "groq/llama-3.1-8b-instant"
print(f"✅ Using model: {MODEL_ID}")

✅ Using model: groq/llama-3.1-8b-instant


In [ ]:
answer_1 = "text"        # 4-letter data format
answer_2 = "interpreter" # the system that runs code
answer_3 = "docstring"   # Python documentation string

print(f"1. LLMs generate: {answer_1}")
print(f"2. Code executed by: {answer_2}")
print(f"3. LLM reads the: {answer_3}")


1. LLMs generate: text
2. Code executed by: interpreter
3. LLM reads the: docstring


In [ ]:
# This is what the LLM "sees" about a tool:
calculator_schema = {
    "type": "function",
    "function": {
        "name": "calculator",
        "description": "Evaluates a mathematical expression",
        "parameters": {
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "Math expression like '2+2' or '100*0.15'"
                }
            },
            "required": ["expression"]
        }
    }
}

print("📋 Tool Schema (what LLM sees):")
print(json.dumps(calculator_schema, indent=2))

📋 Tool Schema (what LLM sees):
{
  "type": "function",
  "function": {
    "name": "calculator",
    "description": "Evaluates a mathematical expression",
    "parameters": {
      "type": "object",
      "properties": {
        "expression": {
          "type": "string",
          "description": "Math expression like '2+2' or '100*0.15'"
        }
      },
      "required": [
        "expression"
      ]
    }
  }
}


In [ ]:
# 🔥 FUNCTION CALLING FLOW - SIMULATED
# This shows EXACTLY what happens when function calling works

def my_calculator(expression):
    """Execute the calculation"""
    try:
        return str(eval(expression))
    except:
        return "Error"

print("🔄 FUNCTION CALLING FLOW")
print("=" * 50)

# Step 1: User asks a question
question = "What is 25% of 840?"
print(f"\n📝 Step 1 - User asks: '{question}'")

# Step 2: LLM generates JSON (this is what a working call produces)
simulated_tool_call = {
    "tool": "calculator",
    "arguments": {"expression": "840 * 0.25"}
}
print(f"\n🤖 Step 2 - LLM generates JSON:")
print(f"   {json.dumps(simulated_tool_call)}")

# Step 3: Framework executes the tool
result = my_calculator(simulated_tool_call["arguments"]["expression"])
print(f"\n⚡ Step 3 - Framework executes tool: {result}")

# Step 4: Result sent back to LLM
print(f"\n✅ Step 4 - Final Answer: 25% of 840 is {result}")

print("\n" + "=" * 50)
print("💡 KEY: The LLM never ran eval() - it just said WHICH")
print("   function to call and WHAT arguments to pass!")

🔄 FUNCTION CALLING FLOW

📝 Step 1 - User asks: 'What is 25% of 840?'

🤖 Step 2 - LLM generates JSON:
   {"tool": "calculator", "arguments": {"expression": "840 * 0.25"}}

⚡ Step 3 - Framework executes tool: 210.0

✅ Step 4 - Final Answer: 25% of 840 is 210.0

💡 KEY: The LLM never ran eval() - it just said WHICH
   function to call and WHAT arguments to pass!


In [ ]:
@tool
def calculator(expression: str) -> str:
    """
    Evaluates a mathematical expression and returns the result.
    Use for arithmetic: addition, subtraction, multiplication, division.

    Args:
        expression: Math like '2+2' or '100*0.15'

    Returns:
        The calculated result as a string.
    """
    try:
        # Only allow safe characters
        allowed = set('____')  # TODO: Fill in: '0123456789+-*/.() '
        if not all(c in allowed for c in expression):
            return "Error: Invalid characters"

        result = ____(expression)  # TODO: Fill in: eval
        return str(result)
    except Exception as e:
        return f"Error: {e}"

print("Tool created!")

Tool created!


In [ ]:
# 🧪 TEST DIRECTLY - No API needed!
print("🧪 Testing calculator:")
print(f"  2 + 2 = {calculator('2 + 2')}")
print(f"  840 * 0.25 = {calculator('840 * 0.25')}")
print(f"  (10 + 5) / 3 = {calculator('(10 + 5) / 3')}")

if calculator('2 + 2') == '4':
    print("\n✅ Correct!")
else:
    print("\n❌ Check your code")

🧪 Testing calculator:
  2 + 2 = Error: Invalid characters
  840 * 0.25 = Error: Invalid characters
  (10 + 5) / 3 = Error: Invalid characters

❌ Check your code


In [ ]:
@tool
def word_counter(text: str) -> str:
    """
    Counts words, characters, and sentences in text.

    Args:
        text: The text to analyze.

    Returns:
        Statistics about the text.
    """
    words = len(text.split())  # TODO: Fill in: split
    chars = len(text)         # TODO: Fill in: len
    sentences = text.count('.') + text.count('!') + text.count('?')

    return f"Words: {words}, Characters: {chars}, Sentences: {sentences}"

print("Tool created!")

Tool created!


In [ ]:
# 🧪 TEST DIRECTLY
print("🧪 Testing word_counter:")
result = word_counter("Hello world. This is a test.")
print(f"  Result: {result}")

if "Words: 6" in result:
    print("\n✅ Correct!")
else:
    print("\n❌ Check your code")

🧪 Testing word_counter:
  Result: Words: 6, Characters: 28, Sentences: 2

✅ Correct!


In [ ]:
@tool
def temperature_converter(value: str, from_unit: str, to_unit: str) -> str:
    """
    Converts temperature between Celsius and Fahrenheit.

    Args:
        value: Temperature value to convert (numeric string).
        from_unit: Source unit, either 'C' for Celsius or 'F' for Fahrenheit.
        to_unit: Target unit, either 'C' for Celsius or 'F' for Fahrenheit.

    Returns:
        Converted temperature as a formatted string with unit.
    """
    try:
        temp = float(value)

        if from_unit.upper() == 'C' and to_unit.upper() == 'F':
            result = temp * 9/5 + 32
        elif from_unit.upper() == 'F' and to_unit.upper() == 'C':
            result = (temp - 32) * 5/9
        else:
            result = temp

        return f"{result:.1f}°{to_unit.upper()}"
    except:
        return "Error: Invalid input"

print("Tool created!")


Tool created!


In [ ]:
# 🧪 TEST DIRECTLY
print("🧪 Testing temperature_converter:")
print(f"  0°C = {temperature_converter('0', 'C', 'F')}")
print(f"  100°C = {temperature_converter('100', 'C', 'F')}")
print(f"  32°F = {temperature_converter('32', 'F', 'C')}")

if "32.0°F" in temperature_converter('0', 'C', 'F'):
    print("\n✅ Correct!")
else:
    print("\n❌ Check: C→F is ×9/5+32, F→C is -32×5/9")

🧪 Testing temperature_converter:
  0°C = 32.0°F
  100°C = 212.0°F
  32°F = 0.0°C

✅ Correct!


In [ ]:
# 🚀 MULTI-TOOL DEMO (Simulated)
# Shows how agent decides which tool to use

# Our available tools (for demo)
tools_demo = {
    "calculator": lambda expr: str(eval(expr)),
    "word_counter": lambda text: f"Words: {len(text.split())}",
    "temperature_converter": lambda v,f,t: f"{float(v)*9/5+32:.1f}°F" if f=='C' else f"{(float(v)-32)*5/9:.1f}°C"
}

# Test cases
test_cases = [
    ("What is 15% of 200?", "calculator", {"expression": "200 * 0.15"}),
    ("How many words in 'Hello world'?", "word_counter", {"text": "Hello world"}),
    ("Convert 25°C to Fahrenheit", "temperature_converter", {"v": "25", "f": "C", "t": "F"})
]

print("🚀 MULTI-TOOL DEMO: Agent Chooses the Right Tool\n")
print("="*60)

for question, expected_tool, args in test_cases:
    print(f"\n❓ Question: {question}")
    print(f"🤖 Agent chooses: {expected_tool}")
    print(f"📋 Arguments: {args}")

    # Execute
    if expected_tool == "calculator":
        result = tools_demo["calculator"](args["expression"])
    elif expected_tool == "word_counter":
        result = tools_demo["word_counter"](args["text"])
    else:
        result = tools_demo["temperature_converter"](args["v"], args["f"], args["t"])

    print(f"✅ Result: {result}")
    print("-"*60)

🚀 MULTI-TOOL DEMO: Agent Chooses the Right Tool


❓ Question: What is 15% of 200?
🤖 Agent chooses: calculator
📋 Arguments: {'expression': '200 * 0.15'}
✅ Result: 30.0
------------------------------------------------------------

❓ Question: How many words in 'Hello world'?
🤖 Agent chooses: word_counter
📋 Arguments: {'text': 'Hello world'}
✅ Result: Words: 2
------------------------------------------------------------

❓ Question: Convert 25°C to Fahrenheit
🤖 Agent chooses: temperature_converter
📋 Arguments: {'v': '25', 'f': 'C', 't': 'F'}
✅ Result: 77.0°F
------------------------------------------------------------


In [ ]:
# Create a ToolCallingAgent (for educational purposes)
model = LiteLLMModel(model_id=MODEL_ID)

agent = ToolCallingAgent(
    tools=[calculator, word_counter],
    model=model,
    max_steps=3,
    verbosity_level=2  # Show reasoning
)

print("✅ ToolCallingAgent created!")
print("\n⚠️ Note: This may fail due to rate limits - that's OK!")

✅ ToolCallingAgent created!

⚠️ Note: This may fail due to rate limits - that's OK!


In [ ]:
# 🚀 OPTIONAL: Test ToolCallingAgent
# This may fail - that's part of learning!

try:
    print("🤖 Testing agent with: 'What is 25 times 4?'")
    print("⏳ (May take a moment or fail...)\n")

    result = agent.run("What is 25 times 4?")
    print(f"\n✅ Result: {result}")

except Exception as e:
    print(f"\n❌ Error (expected): {e}")
    print("\n💡 This is why we test tools DIRECTLY first!")
    print("   The simulated demos above show the concept clearly.")

🤖 Testing agent with: 'What is 25 times 4?'
⏳ (May take a moment or fail...)



╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is 25 times 4?                                                                                             │
│                                                                                                                 │
╰─ LiteLLMModel - groq/llama-3.1-8b-instant ──────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
ModelResponse(id='chatcmpl-6dba75c3-841d-4809-b16d-edc8970730eb', created=1769784681, model='llama-3.1-8b-instant',
object='chat.completion', system_fingerprint='fp_9ca2574dca', choices=[Choices(finish_reason='tool_calls', index=0,
message=Message(content=None, role='assistant',                                                                    
tool_calls=[ChatCompletionMessageToolCall(function=Function(arguments='{"expression":"25*4"}', name='calculator'), 
id='aptn70gq6', type='function')], function_call=None, provider_specific_fields=None))],                           
usage=Usage(completion_tokens=13, prompt_tokens=1319, total_tokens=1332, completion_tokens_details=None,           
prompt_tokens_details=None, queue_time=0.068636033, prompt_time=0.083368965, completion_time=0.009630014,          
total_time=0.092998979), usage_breakdown=None, x_groq={'id': 'req_01kg7p69acfd39c86hrajms57j', 'seed': 678347310}, 
service_tier='auto')                                                                                               

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'calculator' with arguments: {'expression': '25*4'}                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Observations: Error: Invalid characters

[Step 3: Duration 0.35 seconds| Input tokens: 2,757 | Output tokens: 31]

Reached max steps.

[Step 4: Duration 0.30 seconds| Input tokens: 3,040 | Output tokens: 113]


✅ Result: It seems that the calculator function is not working as expected. Let me try a simpler approach.

25 times 4 is equal to 25 multiplied by 4.

In simple multiplication, we multiply the numbers together:
25 * 4 = 25 * 3 + 25 * 1
= 75 + 25
= 100

The answer to the task is: 100.


In [ ]:
@tool
def safe_divide(numerator: str, denominator: str) -> str:
    """
    Safely divides two numbers.

    Args:
        numerator: The top number
        denominator: The bottom number

    Returns:
        The result or an error message.
    """
    try:
        num = float(numerator)
        den = float(denominator)

        # TODO: Check for division by zero
        if den == ____:  # What value causes division by zero?
            return "Error: Cannot divide by zero"

        return str(num / den)
    except ValueError:
        return "Error: Invalid number format"
    except Exception as e:
        return f"Error: {e}"

print("Tool created!")

Tool created!


In [ ]:
# 🧪 TEST ERROR HANDLING
print("🧪 Testing error handling:")
print(f"  10 / 2 = {safe_divide('10', '2')}")
print(f"  10 / 0 = {safe_divide('10', '0')}")
print(f"  abc / 2 = {safe_divide('abc', '2')}")

if "Cannot divide by zero" in safe_divide('10', '0'):
    print("\n✅ Error handling works!")
else:
    print("\n❌ Check your zero check (den == 0)")

🧪 Testing error handling:
  10 / 2 = Error: name '____' is not defined
  10 / 0 = Error: name '____' is not defined
  abc / 2 = Error: Invalid number format

❌ Check your zero check (den == 0)


In [ ]:
# Your custom tool here!
@tool
def my_custom_tool(input_text: str) -> str:
    """
    TODO: Describe what your tool does.

    Args:
        input_text: What the user provides.

    Returns:
        What your tool returns.
    """
    # Your code here!
    return "TODO: Implement me!"

# Test it!
print(my_custom_tool("test"))

TODO: Implement me!
